# Hyperparameter Optimization

TODO: update

## Google Colab

The first cell will only be run in Google Colab, the second one locally as well.

In [1]:
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # noinspection PyUnresolvedReferences
    from google.colab import drive
    drive.mount('/content/drive')

In [2]:
import os
import glob

# let's keep this cell at the beginning for every notebook
# for more convenient training in Google Colab
def get_root_path(filename: str) -> str: 
    """Get root path based on notebook's name."""
    filepath = glob.glob(os.getcwd() + '/**/' + filename, recursive = True)[0]
    return os.path.dirname(os.path.dirname(filepath))

ROOT_PATH = get_root_path('backbones.ipynb')
sys.path.append(ROOT_PATH)

# go to the drive directory
os.chdir(ROOT_PATH) if IN_COLAB else None

## Imports

In [3]:
import os
import cv2
import torch

import albumentations as A
import pandas as pd
import segmentation_models_pytorch as smp
import flaml

from ray import tune
from scripts.evaluation import EvaluationMonitor, get_best_f1_per_setup
from scripts.preprocessing import RoadDataset, split_data
from scripts.training import train_model, setup_seed
from scripts.plotting import plot_post_processing
from torch.utils.data import DataLoader, SubsetRandomSampler

In [ ]:
# necessary for downloading some of the models
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [130]:
setup_seed(16)

## Data

In [131]:
# specify train directory
train_directory = os.path.join(ROOT_PATH, 'data', 'raw', 'train')

In [133]:
# define transformations
train_tf = A.Compose([
    A.Flip(p=0.25),
    A.Rotate(p=0.5, limit=180, border_mode=cv2.BORDER_CONSTANT, rotate_method="ellipse"),
    A.CoarseDropout(max_holes=8, max_height=50, max_width=50, min_holes=None, min_height=25, min_width=25)
])

valid_tf = A.Compose([A.Resize(height=608, width=608, always_apply=True)])

In [ ]:
image_path_train, image_path_val, mask_path_train, mask_path_val = split_data(train_directory, 0.1)
# get train and val dataset instances
train_dataset = RoadDataset(image_path_train, mask_path_train, train_tf)
val_dataset = RoadDataset(image_path_val, mask_path_val, valid_tf)

## Encode-Decoder Combinations



In [134]:
# specify the root path for evaluation json-s
evaluation_dir = os.path.join(ROOT_PATH, 'data', 'results', 'backbone')
monitor = EvaluationMonitor(evaluation_dir)
ENC_DEC_COMBINATIONS = monitor.get_not_updated_models()

## Hyperparameters

Since our aim is to see, how different architectures influence the training, we fix the model, epochs and batch sizes.

In [135]:
ENCODER_WEIGHTS = 'imagenet'

SEED = 16
BATCH_SIZE = 4
N_CPU = os.cpu_count()
N_EPOCHS = 150

## Cross-Validation

In [136]:
def tune_hyperparams(config, encoder: str, decoder: str, datasets: tuple, checkpoint_dir=None):
    
    train_dataset, val_dataset = datasets

    # Create training and validation loaders by providing current K-Fold train/validation indices to Sampler
    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"])
    valid_loader = DataLoader(val_dataset, batch_size=config["batch_size"])

    # Initialize model
    model_ = smp.create_model(decoder, encoder_name=encoder, encoder_weights=ENCODER_WEIGHTS)
    criterion_ = config["criterion"]
    optimizer_ = torch.optim.Adam(model_.parameters(), config["lr"])
    scheduler_ = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer_,
        T_max=(len(train_loader.dataset) * N_EPOCHS) // train_loader.batch_size,
    )
    
    if checkpoint_dir:
        checkpoint = os.path.join(checkpoint_dir, "checkpoint")
        model_state, optimizer_state = torch.load(checkpoint)
        model_.load_state_dict(model_state)
        optimizer_.load_state_dict(optimizer_state)

    train_losses, valid_losses, train_f1s, valid_f1s = train_model(
        model_, (train_loader, valid_loader), criterion_, optimizer_, scheduler_, config["num_epochs"]
    )
    
    tune.report(loss=(val_loss / val_steps), accuracy=correct / total)

In [141]:
max_num_epoch = 150
criteria = [
    smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True),
    smp.losses.FocalLoss(smp.losses.BINARY_MODE)
]

config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "num_epochs": tune.loguniform(1, max_num_epoch),
    "batch_size": tune.randint(1, 5),    # log transformed with base 2
    "criterion": criteria
}